# TRG-73014 UniversalFeatures export compare
Сравнение результатов оригинального экспорта и нового

In [1]:
import pprint
from pprint import pformat

import os
import datetime

from operator import and_
from collections import defaultdict

import six
import luigi
import pyspark.sql.functions as sqlfn

import json
import itertools as it

from pyspark.sql.types import MapType, ArrayType, FloatType, StringType, NumericType

if six.PY3:
    from functools import reduce  # make flake8 happy

In [ ]:
# pprint.pprint(dict(os.environ), width=1)
def log(obj, msg=""):
    if msg: print(msg)
    print("type: {}\ndata: {}".format(type(obj), pformat(obj, indent=1, width=1)))

log(os.environ, "os.environ")
print()
log(dict(os.environ), "dict(os.environ)")

In [3]:
import os
import sys

from pyspark.sql import SparkSession, SQLContext

# Pack executable prj conda environment into zip
TMP_ENV_BASEDIR = "tmpenv"  # Reserved directory to store environment archive
env_dir = os.path.dirname(os.path.dirname(sys.executable))
env_name = os.path.basename(env_dir)
env_archive = "{basedir}/{env}.zip#{basedir}".format(basedir=TMP_ENV_BASEDIR, env=env_name)
os.environ["PYSPARK_PYTHON"] = "{}/{}/bin/python".format(TMP_ENV_BASEDIR, env_name)

# you need this only first time!
# !rm -rf {TMP_ENV_BASEDIR} && mkdir {TMP_ENV_BASEDIR} && cd {TMP_ENV_BASEDIR} && rsync -a {env_dir} . && zip -rq {env_name}.zip {env_name}


In [ ]:
# Create Spark session with prj conda environment and JVM extensions
# `spark-submit ... --driver-java-options "-Dlog4j.configuration=file:/home/vlk/driver_log4j.properties"`
# spark.driver.extraJavaOptions
# 1000 GB => 4000 part. => 8000 real part.

# part_multiplier = 1  # dev
part_multiplier = 16  # prod

spark = (
SparkSession.builder
    .master("yarn-client")
    .appName("TRG-73014-test")
    .config("spark.yarn.queue", "dev.other.regular")
    .config("spark.executor.instances", "2")
    .config("spark.executor.memory", "8G")
    .config("spark.executor.cores", "6")
    .config("spark.executor.memoryOverhead", "2G")
    .config("spark.sql.shuffle.partitions", part_multiplier * 1024)
    .config("spark.driver.memory", "4G")
    .config("spark.driver.maxResultSize", "1G")
    .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:/home/vlk/driver_log4j.properties")
    .config("spark.speculation", "true")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "2")
    .config("spark.dynamicAllocation.maxExecutors", "512")
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")
    .config("spark.network.timeout", "800s")
    .config("spark.reducer.maxReqsInFlight", "10")
    .config("spark.shuffle.io.retryWait", "60s")
    .config("spark.shuffle.io.maxRetries", "10")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer.max", "1024m")
    .config("hive.exec.dynamic.partition", "true")
    .config("hive.exec.dynamic.partition.mode", "nonstrict")
    .config("hive.exec.max.dynamic.partitions", "1000000")
    .config("hive.exec.max.dynamic.partitions.pernode", "100000")
    .config("spark.yarn.dist.archives", env_archive)
    .getOrCreate()
)
#     .config("spark.jars", "hdfs:/lib/prj-transformers-assembly-dev-1.5.0.jar")
sql_ctx = SQLContext(spark.sparkContext)
(spark, sql_ctx)

In [5]:
# end of env. setup

In [ ]:
import os
import numpy as np

from pprint import pformat

import luigi
import pyspark.sql.functions as sqlfn

from pyspark.storagelevel import StorageLevel
from pyspark.sql import DataFrame, SQLContext
from pyspark.sql.types import (
    MapType, ArrayType, FloatType, DoubleType, StringType, StructType, IntegralType, IntegerType
)
from pyspark.sql.utils import CapturedException
from pyspark.ml.wrapper import JavaWrapper

from luigi.contrib.hdfs import HdfsTarget

from prj.apps.utils.common import add_days
from prj.apps.utils.common.hive import format_table, select_clause
from prj.apps.utils.common.luigix import HiveExternalTask
from prj.apps.utils.control.luigix.task import ControlApp
from prj.apps.utils.control.client.exception import FailedStatusException, MissingDepsStatusException

from prj.apps.utils.common import unfreeze_json_param
from prj.apps.utils.common.fs import HdfsClient
from prj.apps.utils.common.hive import FindPartitionsEngine
from prj.apps.utils.common.spark import prjUDFLibrary
from prj.apps.utils.common.luigix import HiveTableSchemaTarget

from prj.apps.utils.common.hive import select_clause
from prj.apps.utils.common.spark import prjUDFLibrary, insert_into_hive
from prj.apps.utils.common.luigix import HiveExternalTask, HiveGenericTarget
from prj.apps.utils.control.luigix import ControlApp, ControlDynamicOutputPySparkTask

In [6]:
# CustomUDFLibrary(spark, "hdfs:/lib/prj-transformers-assembly-dev-1.5.0.jar").register_all_udf()

In [7]:
def show(df, message="dataframe", nlines=20, truncate=False, heavy=True):
    if heavy:
        print("\n{}:".format(message))
        df.printSchema()
        return df

    print("\n{}, rows: {}:".format(message, df.count()))
    df.printSchema()
    df.show(nlines, truncate)
    return df

## UniversalFeatures compare emulation

In [8]:
# gdfs head -n 3 /export/target/universal_features/dm_extended_features_v2/2021-12-31/part-00000-85c2b8eb-591a-4d44-9ff7-a6c198a7191e-c000.csv
old_file_text = """
user;num:ext_feature2
vk:601804359;0:0.133850812912,1:4723.9375,2:0.869369387627,3:0.10000000149,4:0.0500000007451,5:0.0500000007451,6:0.0500000007451,10:3.40000009537,11:5.40000009537,12:2.59999990463,13:15385.1669922,14:8117.16650391,15:203079.578125,16:242712.25,17:320989.59375,18:1333384.875,19:13013.25,20:21976.9160156,21:13660.1669922,22:1720.16662598,23:26525.25,24:0.0434782616794,25:0.0434782616794,26:0.0434782616794,27:0.0869565233588,28:0.0434782616794,29:0.0869565233588,30:0.0434782616794,31:1583.72814941,32:44.6000022888,33:114.233337402,34:393.619049072,35:2361.46655273,36:0.0625,37:0.883668005466,38:23392.9667969,1330:3,1335:3,1343:3,1347:3,1097:3,1243:3,1548:3,1533:3
vk:72293765;0:0.374163866043,1:405.89855957,2:0.920303463936,3:0.0774647891521,4:0.0211267601699,5:0.0140845067799,6:0.0845070406795,7:0.866715550423,8:0.709127545357,9:0.510675251484,10:5.32258081436,11:1.13709676266,12:3.30645155907,13:15810.6582031,14:8899.97558594,15:197297.53125,16:223353.984375,17:317720.75,18:1335009.125,19:12406.796875,20:21937.2792969,21:9697.89746094,22:1478.60449219,23:23335.1816406,24:0.0206896550953,25:0.00689655169845,26:0.00689655169845,27:0.110344827175,28:0.00689655169845,29:0.0206896550953,30:0.00689655169845,31:1944.44030762,32:45.0097999573,33:135.024597168,34:742.001220703,35:2077.11474609,36:0.253623187542,37:0.919775247574,38:21882.328125,1547:452,2120:3,1097:2421,1122:25,2127:1,2130:3,2131:3,2132:3,2138:208,2145:498,2146:8,2147:3,2066:5,2070:9,2187:2,2188:18,2157:1,2212:7,2213:5,2290:5,2218:1,2224:1,2231:6,2232:35,2233:53,2237:2,2241:2,2245:78,1057:12,2248:7,2259:1,2261:8,1243:73,1247:1651,1062:132,2278:1,2283:1,2284:10,2285:1,2287:1,2288:1,1266:144,2291:5,2295:5,2296:3,2305:16,2307:1,2310:2,2316:19,2318:2,2319:14,2321:2,1302:6,2333:60,2334:12,2335:33,1319:734,2345:24,2346:2,2348:2,2350:1,1330:2593,1333:15,2358:6,1335:2593,1337:15,1338:491,2363:2,1340:243,1343:1874,1346:33,1347:1723,1348:452,2359:2,2384:1,2413:24,2415:2,2417:119,2418:2,2419:20,2426:233,2428:2,2429:69,2430:80,2431:1,1426:33,1548:1769,1453:73,2379:2,2012:1,2016:1,2021:3,2022:35,2023:2,2024:1,2029:1,2036:3,2217:1,1533:1271
ok:571813119726;1547:20,1161:10,1345:20,1335:20,1343:20,1051:10,1330:20.01,1061:10.0,1253:20.0,1348:20.0
"""
# gdfs head -n 3 /data/prj/dev/apps/export/universal_features/dm_extended_features_v2/2022-01-09/part-00000-1929771c-58d1-4a7a-938b-775fc92beaae-c000.csv
new_file_text = """
user;num:ext_feature2
ok:571813119726;1547:20.00001,1161:10.0,1345:20.0,1335:20.0,1343:20.0,1051:10.0,1330:20.0,1061:10.0,1253:20.0,1348:20.0
ok:590558313987;12:0.0,1547:48.0,8:0.40449977,4:0.2,1221:10.0,11:1.0,9:0.40449977,1323:14.0,26:0.125,37:0.24138813,24:0.125,1265:23.0,5:0.2,10:0.0,1345:48.0,1335:48.0,6:0.2,1320:1.0,36:0.0,1343:38.0,1:2792.0,25:0.125,1062:32.0,1051:16.0,0:0.6542008,27:0.125,2:0.24138813,1330:48.0,30:0.125,7:0.40449977,29:0.125,1338:10.0,3:0.2,28:0.125,1348:48.0
"""

In [9]:
def drop_empty_lines(text):
    lines = text.split("\n")
    lines = [x.strip() for x in lines if len(x.strip()) > 0]
    return "\n".join(lines)

In [10]:
def write_text(text, file_path):
    with open(file_path, 'w') as f:
        f.write(text)

In [11]:
def save_to_hdfs(text, hdfs_dir, file_name):
    from backports import tempfile
    from prj.apps.utils.common.fs import HdfsClient
    hdfs = HdfsClient()
    success_filename = "_SUCCESS"

    with tempfile.TemporaryDirectory() as tmp_local_dir:
        write_text(drop_empty_lines(text), os.path.join(tmp_local_dir, file_name))
        hdfs.mkdir(hdfs_dir, remove_if_exists=True)
        hdfs.put(tmp_local_dir, hdfs_dir, content_only=True)
    hdfs.touchz(os.path.join(hdfs_dir, success_filename))

In [12]:
save_to_hdfs(old_file_text, "hdfs:/user/vlk/test/TRG-73014/dm_extended_features_v2/old/", "features.csv")
save_to_hdfs(new_file_text, "hdfs:/user/vlk/test/TRG-73014/dm_extended_features_v2/new/", "features.csv")

## End of emulation, pay attention!

In [8]:
# path="hdfs:/user/vlk/test/TRG-73014/dm_extended_features_v2/old/"
path="hdfs:/export/target/universal_features/dm_extended_features_v2/2021-12-31/"

old_lines_df = show(spark.read.csv(
    path=path,
    header=True,
    sep=";"
))
# .persist(StorageLevel.MEMORY_ONLY)


dataframe:
root
 |-- user: string (nullable = true)
 |-- num:ext_feature2: string (nullable = true)



In [9]:
# path="hdfs:/user/vlk/test/TRG-73014/dm_extended_features_v2/new/"
path="hdfs:/data/prj/dev/apps/export/universal_features/dm_extended_features_v2/2022-01-09/"

new_lines_df = show(spark.read.csv(
    path=path,
    header=True,
    sep=";"
))
# .persist(StorageLevel.MEMORY_ONLY))


dataframe:
root
 |-- user: string (nullable = true)
 |-- num:ext_feature2: string (nullable = true)



In [10]:
joined = show(
    old_lines_df.alias("a")
    .join(new_lines_df.alias("b"), ["user"])
    .selectExpr("user", "a.`num:ext_feature2` as a_f", "b.`num:ext_feature2` as b_f")
)


dataframe:
root
 |-- user: string (nullable = true)
 |-- a_f: string (nullable = true)
 |-- b_f: string (nullable = true)



In [11]:
def _format(item):
    k, v = item.split(":")    
    v = str(int(float(v) * 100000))  # 5 fraction digits for float32 # compare with relative tolerance is much better
    return "{}:{}".format(k, v[:6])  # only first 6 digits should be compared

def _normalize(text):
    items = text.split(",")
    formatted_items = [_format(x) for x in items]
    return sorted(formatted_items, key=lambda x: int(x.split(":")[0]))

def _diff(a, b):
    a_items = _normalize(a)
    b_items = _normalize(b)
    if a_items == b_items:
        return None, None

    if len(a_items) != len(b_items):
        return -1, 100500.0  # key = -1 if len is different

    for i, (ai, bi) in enumerate(zip(a_items, b_items)):
        ak, av = ai.split(":")
        bk, bv = bi.split(":")
        if ak != bk:
            return int(ak), float(bk)  # diff = key(b) if keys is defferent

        diff = abs(int(av) - int(bv))
        if diff > 1:
            return int(ak), float(diff) / 10000.0  # key(a), abs_diff(val(a), val(b))

    return None, None  # if a == b

def transform(rows):
    for row in rows:
        key, diff = _diff(row["a_f"], row["b_f"])
        yield row["user"], key, diff, row["a_f"], row["b_f"]

diff = spark.createDataFrame(
    joined.rdd.mapPartitions(transform, preservesPartitioning=True),
    schema="user:string,key:int,diff:float,a_f:string,b_f:string"
)

In [12]:
dir_path = "hdfs:/user/vlk/test/TRG-73014/dm_extended_features_v2/old_new_diff/"

diff.where(
    "diff is not null"
).write.option(
    "mapreduce.fileoutputcommitter.algorithm.version", "2"
).parquet(
    dir_path, 
    mode="overwrite"
)

diff = spark.read.parquet(dir_path).cache()
show(diff, heavy=False)


dataframe, rows: 0:
root
 |-- user: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- diff: float (nullable = true)
 |-- a_f: string (nullable = true)
 |-- b_f: string (nullable = true)

+----+---+----+---+---+
|user|key|diff|a_f|b_f|
+----+---+----+---+---+
+----+---+----+---+---+



DataFrame[user: string, key: int, diff: float, a_f: string, b_f: string]

In [13]:
diff.unpersist()
spark.stop()